# AUDIT-C Final Data Prep

In [ ]:
%matplotlib inline

import datetime as dt
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
from tqdm import tqdm

init_notebook_mode(connected=True)
InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

tqdm.pandas()

In [ ]:
v1sid, v2sid =  'syn10250481', 'syn9974011'

v1r = pd.read_excel(syn.get(v1sid).path, parse_dates=['response_utc'])
v2r = pd.read_csv(syn.get(v2sid).path, parse_dates=['createdAt'])

v1r.head()
v2r.head()

### Process V1 data

In [ ]:
# drop the uneeded columns
v1 = v1r.drop(columns=[
    'sent_time_local', 'sent_time_utc', 'response_local', 'response_id', 'user_id', 'audit'
]).rename(columns={
    'brightenid': 'participant_id',
    'audit1':'alc_1',
    'audit2':'alc_2',
    'audit3':'alc_3',
    'response_utc':'dt_response'
})

# add qsum
v1['alc_sum'] = [t.alc_1 + t.alc_2 + t.alc_3 for t in v1.itertuples()]

v1.head()

### Process V2 data

In [ ]:
v2 = v2r.drop(columns=['day'])\
     .rename(columns={
        'How often did you have a drink containing alcohol in the past year?':'alc_1',
        'How many drinks did you have on a typical day when you were drinking in the past year?':'alc_2',
        'How often did you have six or more drinks on one occasion in the past year?':'alc_3',
        'username':'participant_id',
        'createdAt': 'dt_response'
})

# add qsum
v2['alc_sum'] = [t.alc_1 + t.alc_2 + t.alc_3 for t in v2.itertuples()]

v2.head()

### Combine the DataFrames

In [ ]:
combined = pd.concat([v1, v2], sort=False)

# order the columns
combined = combined.loc[:,
    ['participant_id', 'dt_response', 'alc_1', 'alc_2', 'alc_3', 'alc_sum']
]

combined.head()

In [ ]:
combined.alc_1.hist()
combined.alc_1.value_counts()

In [ ]:
combined.alc_2.hist()
combined.alc_2.value_counts()

In [ ]:
combined.alc_3.hist()
combined.alc_3.value_counts()

### Add week into study

In [ ]:
metasid = 'syn17023349'
metadata = syn.tableQuery(f'SELECT participant_id, startdate FROM {metasid}').asDataFrame(convert_to_datetime=True)
metadata.startdate = pd.to_datetime(metadata.startdate)

# add in the participants start date as a new column
combined = pd.merge(combined, metadata, on='participant_id', how='left')

# get the time difference in weeks as a float
combined['week'] = [
    d.days/7 for d in (
        combined.dt_response.apply(
            lambda x: dt.datetime(year=x.year, month=x.month, day=x.day))-combined.startdate
    )
]

# convert the week number to an int by taking the floor
combined.week = combined.week.progress_apply(lambda x: np.int16(np.floor(x))+1)# if not pd.isnull(x) else np.nan)

# remove the start date
combined = combined.drop(columns=['startdate'], errors='ignore')

# reorder the columns
cols = list(combined.columns)
cols = cols[0:2] + ['week'] + cols[2:-1]
combined = combined.reindex(columns=cols)

combined.head()

### Localize timestamps

In [ ]:
# localize timestamps
combined['dt_response'] = [
    str(t.tz_localize('UTC'))
    for t in combined.dt_response
]

combined.head()

### Set provenance and upload to Synapse

In [ ]:
t = syn.delete(
    syn.tableQuery('select * from syn17021280')
)

In [ ]:
final = syn.store(Table(
    Schema(
            name='AUDIT-C',
            columns=as_table_columns(combined), 
            parent='syn10848316'),
        combined
    )
)

In [ ]:
final = syn.setProvenance(
    'syn17021280',
    activity=Activity(
        name='Combine V1 and V2 data',
        description='Process and combine the data collected during study 1 and study 2',
        used=[v1sid, v2sid],
        executed=[
            dict(
                name='IPython Notebook',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_Alcohol_datafiles.ipynb'
            )
        ]
    )
)